# Building with LangChain

In [2]:
import os
import sys
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

load_dotenv()  # Load API keys and other sensitive data from the .env file

True

In [3]:
api_key = os.getenv("APIKEY")

# LLM Chain

In [4]:
llm = ChatOpenAI(openai_api_key=api_key)  # Create the ChatGPT instance

In [5]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content="Langsmith can help with testing by providing automated testing tools and frameworks that can be used to create and run tests on software applications. These tools can help streamline the testing process, identify bugs and issues, and ensure that the software is functioning correctly. Additionally, Langsmith's testing capabilities can help improve code quality, reduce the risk of errors, and ultimately lead to a more reliable and stable software product.")

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [9]:
chain = prompt | llm 

In [10]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='Langsmith can help with testing in various ways by providing tools and frameworks that streamline the testing process and improve the efficiency and effectiveness of testing activities. Some ways Langsmith can assist with testing include:\n\n1. Automated Testing: Langsmith offers libraries and modules for automated testing, allowing testers to easily create and run automated test scripts to validate the functionality of their software. This helps in reducing manual effort and improving test coverage.\n\n2. Test Data Generation: Langsmith can be used to generate test data for different scenarios, making it easier for testers to simulate various conditions and edge cases during testing. This ensures comprehensive test coverage and helps in identifying potential issues early in the development cycle.\n\n3. Performance Testing: Langsmith provides tools for performance testing, allowing testers to measure the performance of their software under different conditions and id

In [11]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [12]:
chain = prompt | llm | output_parser

In [13]:
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can help with testing in several ways:\n\n1. Automated Testing: Langsmith can generate test cases based on the grammar rules provided, helping to automate the testing process and ensure comprehensive test coverage.\n\n2. Test Data Generation: Langsmith can generate test data for different types of inputs, helping to validate the functionality and performance of the system under various scenarios.\n\n3. Boundary Testing: Langsmith can generate test cases that cover boundary conditions, helping to identify potential issues related to input validation and boundary handling.\n\n4. Regression Testing: Langsmith can help in generating regression test cases to ensure that new code changes do not introduce any unintended side effects or break existing functionality.\n\nOverall, Langsmith can be a valuable tool for enhancing the efficiency and effectiveness of testing processes by providing automated test case generation and test data generation capabilities.'

# Retrieval Chain

In [14]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")

docs = loader.load()

In [15]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=api_key)

In [16]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [17]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [18]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help visualize test results.'

In [19]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [20]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith can help with testing by providing the ability to debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework. It offers tracing and evaluation capabilities, as well as a Prompt Hub for managing prompts. Additionally, LangSmith seamlessly integrates with LangChain, the open source framework for building with LLMs, to further support testing efforts.


# Conversation Retrieval Chain

In [21]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [22]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith supports at each stage of the LLM application lifecycle.Setup: Learn 

In [23]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [24]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following s

# Agent